In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from transformers import pipeline
import jieba
from gensim import corpora, models
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import joblib 
from joblib import loadrom sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold
from xgboost import XGBRegressor

import optuna

/Users/macbookair/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [26]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [6]:
data_train=pd.read_csv("/Users/macbookair/Documents/python 2/data.csv")
data_train_price=data_train["价格"]
data_train=data_train.drop(columns=["Unnamed: 0","价格","text"])
data_pred=pd.read_csv("/Users/macbookair/Documents/python 2/X_pred.csv")
data_pred=data_pred.drop(columns=["Unnamed: 0","text"])

In [7]:
#文本的情感分析 使用在hugging face上运用大众点评数据进行调节的Bert 进行情感分析
model_path= "/Users/macbookair/Documents/transformer"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

def text_score(text,tokenizer=tokenizer,model=model):
    # 编码并推理
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)#将原始文本转数值输入
    with torch.no_grad():
        outputs = model(**inputs)#正向传播
        probs = F.softmax(outputs.logits, dim=1)#将分类问题通过softmax转化为概率
    # 获取概率
    positive_prob = probs[0][1].item()
    negative_prob = probs[0][0].item()

    # 自定义评分（线性映射）
    score = positive_prob * 5 + negative_prob * 1
    return score

/Users/macbookair/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [8]:
data_train[score]=data_train.apply(lambda row:text_score(row["text"]),axis=1 )

In [7]:
X=data_train
y=data_train_price
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
#因为样本数量太多 所以随机选择10000进行调参
sample_size = 10000
if len(X) > sample_size:
    np.random.seed(42)  # 保证可复现
    indices = np.random.choice(len(X), sample_size, replace=False)
    X_sample = X.iloc[indices,:]
    y_sample = y[indices]
else:
    X_sample = X
    y_sample = y
X__sampletrain,X_sample_test,y_sample_train,y_sample_test=train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

In [9]:
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
    }

    model = XGBRegressor(**param)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    return mse 

In [26]:
study = optuna.create_study(direction='minimize')  # 回归是“最小化”损失
study.optimize(objective, n_trials=50)
best_params = study.best_params

In [10]:
best_par={'booster': 'dart',
 'lambda': 0.008244759581148755,
 'alpha': 0.6405598303602479,
 'subsample': 0.9721516428059781,
 'colsample_bytree': 0.6368787772510718,
 'learning_rate': 0.15427365499198334,
 'max_depth': 5,
 'n_estimators': 154}

In [28]:
# emsemble 集成学习。将ridge randomforest 和 XGboost 作为第一层模型
models = {
    'ridge': Ridge(alpha=1.0),
    'rf': RandomForestRegressor(n_estimators=100, random_state=42),
    'xgb': XGBRegressor(**best_par),
    'catboost': CatBoostRegressor(n_estimators=150, learning_rate=0.1, depth=6, verbose=0, random_state=42),
    'lgbm': LGBMRegressor(n_estimators=150, learning_rate=0.1, max_depth=6, random_state=42)
}  
# 3. 训练第一层模型并生成折外预测
kf = KFold(n_splits=5, shuffle=True, random_state=42)
#存储每个模型在训练集上的 （Out-of-Fold）预测
meta_features_train = np.zeros((X_train.shape[0], len(models)))
# 存储每个模型对验证集 X_val 的平均预测结果
meta_features_val = np.zeros((X_test.shape[0], len(models)))

for i, (name, model) in enumerate(models.items()):
    print(f"Training {name}...")
    oof_predictions = np.zeros(X_train.shape[0])
    val_predictions = np.zeros(X_test.shape[0])
    
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model.fit(X_tr, y_tr)
        oof_predictions[val_idx] = model.predict(X_val_fold)
        val_predictions += model.predict(X_test) / kf.n_splits
    joblib.dump(model, f"{name}_model.pkl")
    
    meta_features_train[:, i] = oof_predictions
    meta_features_val[:, i] = val_predictions
# 4. 训练元模型
meta_model = XGBRegressor(n_estimators=60, learning_rate=0.1, random_state=42)
meta_model.fit(meta_features_train, y_train)
joblib.dump(meta_model, "meta_model.pkl")

# 5. 预测和评估
test_predictions = np.zeros((X_test.shape[0], len(models)))
for i, (name, model) in enumerate(models.items()):
    test_predictions[:, i] = model.predict(X_test)

meta_test_predictions = meta_model.predict(test_predictions)
final_predictions = meta_test_predictions  

rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
print(f"Test RMSE: {rmse}")

Training ridge...
Training rf...
Training xgb...
Training catboost...
Training lgbm...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024027 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1069
[LightGBM] [Info] Number of data points in the train set: 53844, number of used features: 90
[LightGBM] [Info] Start training from score 1956461.644566
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

In [53]:
def objective(weights):
    weights = np.array(weights)
    weights /= np.sum(weights)  # 归一化权重
    final_pred = np.sum([w * pred for w, pred in zip(weights, test_predictions.T)], axis=0)
    return mean_squared_error(y_test, final_pred, squared=False)  # RMSE

# 初始权重 6 个，对应 5 个基础模型 + 1 个 meta model
initial_weights = [1/6] * 6  
bounds = [(0, 1)] * 6

result = minimize(objective, initial_weights, bounds=bounds)
optimal_weights = result.x / np.sum(result.x)

print(" Optimal Voting Weights (with meta model):", optimal_weights)

 Optimal Voting Weights (with meta model): [0.         0.49579131 0.50420869 0.         0.         0.        ]


In [50]:
#predict
models = {
    'ridge': load('Ridge_model.pkl'),
    'rf': load('rf_model.pkl'),
    'xgb': load('xgb_model.pkl'),
    "catboost": load('catboost_model.pkl'),
    'lgbm': load('lgbm_model.pkl')
}
meta_model = load('meta_model.pkl')
# 替换为你的实际优化权重（确保与训练时一致）
voting_weights = {'ridge':0,'rf':0.49579131,"xgb":0.50420869,"catboost":0,'lgbm':0,"meta_model":0}
total_weight = sum(voting_weights.values())
voting_weights = {k: v / total_weight for k, v in voting_weights.items()}
print("Normalized Voting Weights:", voting_weights)

Normalized Voting Weights: {'ridge': 0.0, 'rf': 0.49579131, 'xgb': 0.50420869, 'catboost': 0.0, 'lgbm': 0.0, 'meta_model': 0.0}


In [51]:
# 第一层模型预测
base_predictions = np.zeros((data_pred.shape[0], len(models)))
for i, (name, model) in enumerate(models.items()):
    print(f"Predicting with {name}...")
    base_predictions[:, i] = model.predict(data_pred)

# 元模型预测
meta_predictions = meta_model.predict(base_predictions)
#voting 得到最后结果
final_predictions = np.zeros(data_pred.shape[0])
for i, name in enumerate(models.keys()):
    final_predictions += voting_weights[name] * base_predictions[:, i]
final_predictions +=voting_weights["meta_model"] * meta_predictions

results = pd.DataFrame({
    'predicted_price': final_predictions
})

Predicting with ridge...
Predicting with rf...
Predicting with xgb...
Predicting with catboost...
Predicting with lgbm...


In [45]:
best_par={'booster': 'dart',
 'lambda': 0.008244759581148755,
 'alpha': 0.6405598303602479,
 'subsample': 0.9721516428059781,
 'colsample_bytree': 0.6368787772510718,
 'learning_rate': 0.15427365499198334,
 'max_depth': 5,
 'n_estimators': 154}

In [52]:
y_test_pred=pd.DataFrame(results)
y_test_pred.to_csv("/Users/macbookair/Documents/results.csv")